In [8]:
import numpy as np
import hashlib
import pickle
import os

from scipy.spatial.transform import Rotation as R
from aist_plusplus.loader import AISTDataset

In [2]:
anno_dir = '/srv/share/datasets/AIST/aist_plusplus_final/'

flag_train = True
flag_val = True
flag_test = True

# create list
seq_names = []
if flag_train:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_train.txt"), dtype=str
    ).tolist()
if flag_val:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_val.txt"), dtype=str
    ).tolist()
if flag_test:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_test.txt"), dtype=str
    ).tolist()

ignore_list = np.loadtxt(
    os.path.join(anno_dir, "ignore_list.txt"), dtype=str
).tolist()
seq_names = [name for name in seq_names if name not in ignore_list]
n_samples = len(seq_names)

n_samples

992

In [3]:
def compute_SMPL_motion(seq_name, motion_dir):
    smpl_poses, smpl_scaling, smpl_trans = AISTDataset.load_motion(motion_dir, seq_name)
    smpl_trans /= smpl_scaling
    smpl_poses = R.from_rotvec(
        smpl_poses.reshape(-1, 3)).as_matrix().reshape(smpl_poses.shape[0], -1)
    smpl_motion = np.concatenate([smpl_trans, smpl_poses], axis=-1)
    return smpl_motion


motion_dir = os.path.join(anno_dir, 'motions')
for i, seq_name in enumerate(seq_names[:1]):
    print("processing %d / %d" % (i + 1, n_samples))

    smpl_motion = compute_SMPL_motion(seq_name, motion_dir)
    print(f"Seq name: {seq_name} \t Motion shape: {smpl_motion.shape}")

processing 1 / 992
Seq name: gWA_sFM_cAll_d25_mWA4_ch05 	 Motion shape: (1919, 219)


In [4]:
seq_name, smpl_motion.shape

('gWA_sFM_cAll_d25_mWA4_ch05', (1919, 219))

## Hash based encoding

In [16]:
h = hashlib.sha1(seq_name.encode('utf-8'))
h_str = h.hexdigest()
seq_name, h, h_str

('gWA_sFM_cAll_d25_mWA2_ch03',
 <sha1 HASH object @ 0x7f9014096d50>,
 'ad14b7cbe9559891300039c34deb35fe41321046')

In [17]:
stride = 4
h_np = [float(int(h_str[i:i+stride], 16)) / 16**stride for i in range(0, len(h_str), stride)]
h_np = np.array(h_np).reshape((1,-1))
h_np, h_np.shape

(array([[0.67608643, 0.71794128, 0.91145325, 0.59596252, 0.1875    ,
         0.22563171, 0.30436707, 0.21090698, 0.25466919, 0.06356812]]),
 (1, 10))

In [18]:
target_shape = (256,smpl_motion.shape[-1])
# target_shape = smpl_motion.shape
target_shape

(256, 219)

In [19]:
op_dim = np.prod(target_shape)
op_dim

56064

In [20]:
wt_seed = 101
wt_rng = np.random.default_rng(seed=wt_seed)

hidden_size = 128

w1 = wt_rng.normal(size=(h_np.shape[-1], hidden_size))
b1 = wt_rng.normal(size=hidden_size)
w2 = wt_rng.normal(size=(hidden_size, op_dim))
b2 = wt_rng.normal(size=op_dim)

z1 = h_np @ w1 + b1
op = np.tanh(z1) @ w2 + b2
op = op.reshape(target_shape)
op.shape

(256, 219)

In [21]:
op

array([[-11.91354572,  -5.7861633 ,  -1.8870801 , ...,  21.59820501,
          1.29898365,   0.20761964],
       [ -0.42679495,  -1.54306516,   6.25812674, ...,  -4.46167475,
          2.63731778,  -3.35751478],
       [  4.11336119,  -5.81576417,   0.21762593, ...,   6.76187817,
         -8.83036263,  -0.12478315],
       ...,
       [ -7.02777121,   2.24370707,  10.21025277, ...,  -7.88181658,
         -1.93101821,   9.23042816],
       [  6.97445448,   0.05338962,  12.24750809, ...,  -6.75456146,
         12.18555258,  10.56377414],
       [  1.08006478, -14.35862257,  10.30874062, ...,  -0.52610094,
         -8.13066487,  -3.35579391]])

In [23]:
def encode(smpl_motion, seq_name, target_shape):
    global w1, b1, w2, b2
    h = hashlib.sha1(seq_name.encode('utf-8'))
    h_str = h.hexdigest()
    
    stride = 4
    h_np = [float(int(h_str[i:i+stride], 16)) / 16**stride for i in range(0, len(h_str), stride)]
    h_np = np.array(h_np).reshape((1,-1))
    
    z1 = h_np @ w1 + b1
    op = np.tanh(z1) @ w2 + b2
    return op.reshape(target_shape)

np.allclose(op, encode(smpl_motion, seq_name, target_shape))

True

In [ ]:
# uniqueness test
target_shape = (256,smpl_motion.shape[-1])
codes = []
for i, seq_name in enumerate(seq_names):
    print("Processing: %d / %d" % (i + 1, n_samples))

    smpl_motion = compute_SMPL_motion(seq_name, motion_dir)
    enc = encode(smpl_motion, seq_name, target_shape)

    uniq = True
    for ii, code in enumerate(codes):
        if np.allclose(code, enc):
            print(f"-- Codes for samples {i+1},{ii+1} same!")
            uniq = False
            break
    if uniq:
        print(f"++ Unique code for sample {i+1}")
        codes.append(enc)

In [25]:
len(codes), len(seq_names)

(992, 992)

## One-hot encoding

In [7]:
subset_size = 20

onehot_enc_map = {}
for i, seq_name in enumerate(seq_names[:subset_size]):
    enc = np.zeros(subset_size)
    enc[i] = 1

    onehot_enc_map[seq_name] = { 'index': i, 'enc': enc }

onehot_enc_map

{'gWA_sFM_cAll_d25_mWA4_ch05': {'index': 0,
  'enc': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.])},
 'gWA_sFM_cAll_d25_mWA2_ch03': {'index': 1,
  'enc': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.])},
 'gWA_sFM_cAll_d27_mWA2_ch21': {'index': 2,
  'enc': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.])},
 'gWA_sFM_cAll_d25_mWA5_ch07': {'index': 3,
  'enc': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.])},
 'gWA_sFM_cAll_d27_mWA2_ch17': {'index': 4,
  'enc': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.])},
 'gWA_sFM_cAll_d25_mWA5_ch06': {'index': 5,
  'enc': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.])},
 'gWA_sFM_cAll_d26_mWA5_ch13': {'index': 6,
  'enc': array([0., 0., 0., 0., 0., 0., 1., 

In [9]:
# pass through small NN

with open('/srv/share4/anarayanan68/mint/_expts/tvloss_overfit_l2_bsz8_1GPU/enc_data.pkl', 'rb') as pf:
    enc_pkl = pickle.load(pf)

enc_pkl

{'w1': array([[-0.7901525 , -2.03462548,  0.60330175, ...,  0.44634342,
         -1.14135364,  0.08615977],
        [ 1.82124749,  0.12384311, -1.20305584, ..., -1.21744209,
         -0.82092046, -0.09132382],
        [-2.2653625 , -0.43572113, -0.98194386, ...,  0.3687278 ,
         -0.84107251, -1.36113085],
        ...,
        [ 0.70120373, -0.8149087 , -1.07677802, ...,  0.02262641,
          0.71992117,  0.96491221],
        [-0.72263791, -0.57635686, -0.31297773, ..., -0.29936611,
          0.40719516, -0.95693612],
        [-2.15101827,  0.38983742,  0.71835953, ..., -1.77863634,
         -1.51518335,  1.0290083 ]]),
 'b1': array([ 0.20061542, -0.35209537, -1.16466148,  0.14945811, -1.41468317,
        -0.24597329,  0.57774547, -0.37396595, -1.68107635, -1.78507783,
        -0.57151754, -2.36868568,  0.66903662, -0.83797974, -0.75988824,
        -0.49501931, -0.90790685,  1.20264581, -0.79971137, -1.35456058,
         0.93480667, -0.84045751,  1.36679149,  0.59832129, -0.558631

In [10]:
target_shape = (256,smpl_motion.shape[-1])
# target_shape = smpl_motion.shape
target_shape

(256, 219)

In [11]:
op_dim = np.prod(target_shape)
op_dim

56064

In [13]:
wt_seed = enc_pkl['wt_seed']
wt_rng = np.random.default_rng(seed=wt_seed)

hidden_size = enc_pkl['hidden_size']

w1 = wt_rng.normal(size=(subset_size, hidden_size))
b1 = wt_rng.normal(size=hidden_size)
w2 = wt_rng.normal(size=(hidden_size, op_dim))
b2 = wt_rng.normal(size=op_dim)

In [14]:
seq_name, onehot_enc_map[seq_name]

('gLH_sBM_cAll_d16_mLH2_ch04',
 {'index': 19,
  'enc': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1.])})

In [15]:
ip = onehot_enc_map[seq_name]['enc']

z1 = ip @ w1 + b1
op = np.tanh(z1) @ w2 + b2
op = op.reshape(target_shape)
op.shape

(256, 219)

In [16]:
op

array([[-3.57312332e-01,  1.39836928e+01,  4.51780611e+00, ...,
        -1.40807913e+00,  5.63295300e+00, -1.37259505e+01],
       [-7.22731946e+00, -1.59952002e+01, -8.68384809e-01, ...,
         8.34062849e-01,  5.72835125e+00,  1.26231974e+01],
       [-8.91031507e+00,  5.67676571e-01, -1.73737162e+01, ...,
        -1.63558135e-03, -4.13480959e+00,  1.26647066e+00],
       ...,
       [-4.06349180e+00, -1.54052084e+00, -1.14557292e+01, ...,
         2.57522258e+00, -5.17123343e+00, -6.93886543e+00],
       [-2.00960292e-01, -1.14891874e+01,  4.04385440e-01, ...,
         2.28700280e+00,  6.05607930e+00,  7.27597370e+00],
       [-2.55571639e+00, -7.09158924e+00,  8.94709391e-01, ...,
         1.13004440e+00, -1.03669216e+01,  3.39689352e+00]])

In [17]:
def encode(seq_name, onehot_enc_map, target_shape):
    global w1, b1, w2, b2
    ip = onehot_enc_map[seq_name]['enc']
    
    z1 = ip @ w1 + b1
    op = np.tanh(z1) @ w2 + b2
    return op.reshape(target_shape)

np.allclose(op, encode(seq_name, onehot_enc_map, target_shape))

True

In [18]:
# uniqueness test
codes = []
for i, seq_name in enumerate(seq_names[:subset_size]):
    print("Processing: %d / %d" % (i + 1, subset_size))

    smpl_motion = compute_SMPL_motion(seq_name, motion_dir)
    enc = encode(seq_name, onehot_enc_map, target_shape)

    uniq = True
    for ii, code in enumerate(codes):
        if np.allclose(code, enc):
            print(f"-- Codes for samples {i+1},{ii+1} same!")
            uniq = False
            break
    if uniq:
        print(f"++ Unique code for sample {i+1}")
        codes.append(enc)

Processing: 1 / 20
++ Unique code for sample 1
Processing: 2 / 20
++ Unique code for sample 2
Processing: 3 / 20
++ Unique code for sample 3
Processing: 4 / 20
++ Unique code for sample 4
Processing: 5 / 20
++ Unique code for sample 5
Processing: 6 / 20
++ Unique code for sample 6
Processing: 7 / 20
++ Unique code for sample 7
Processing: 8 / 20
++ Unique code for sample 8
Processing: 9 / 20
++ Unique code for sample 9
Processing: 10 / 20
++ Unique code for sample 10
Processing: 11 / 20
++ Unique code for sample 11
Processing: 12 / 20
++ Unique code for sample 12
Processing: 13 / 20
++ Unique code for sample 13
Processing: 14 / 20
++ Unique code for sample 14
Processing: 15 / 20
++ Unique code for sample 15
Processing: 16 / 20
++ Unique code for sample 16
Processing: 17 / 20
++ Unique code for sample 17
Processing: 18 / 20
++ Unique code for sample 18
Processing: 19 / 20
++ Unique code for sample 19
Processing: 20 / 20
++ Unique code for sample 20


In [19]:
len(codes), len(seq_names[:subset_size])

(20, 20)

In [21]:
codes[0] - codes[1]

array([[ -3.7172471 ,   0.7417453 ,  -7.74859264, ...,  -8.83078509,
         -9.78254557,  -0.19112167],
       [  0.45259177,  -7.82144513,  -5.49666808, ...,  13.25936315,
         -7.17913427,  -9.03646386],
       [  8.67022638,   9.38052127,   3.58080637, ...,   5.25710151,
         11.70754968,  -3.59840717],
       ...,
       [ -3.81926009,   1.77586849, -16.97687696, ...,  11.44402348,
         -3.42204133,   1.78174752],
       [ -7.39002791,  -0.84626922, -25.00937199, ...,   0.44423946,
         -1.72451307,  -0.76579427],
       [  3.63432833,   4.77108345,   5.60641838, ...,  12.4190103 ,
         14.31144562,   1.97608226]])